In [1]:
# Cell 1 — Setup & imports

import os, glob, time, random
from datetime import datetime

import numpy as np
import pandas as pd
import networkx as nx

# Project helpers you already have
from helpers import (
    load_fixeddepth_summary, parse_fn, load_runs, make_key
)
from graph_utils import load_network, load_abm_network, get_individuals_from_graph

from pathlib import Path
import os

repo_root = Path.cwd().parent          
data_dir  = repo_root / "data"

os.environ["DTU_CSV"]        = str(data_dir / "bt_symmetric.csv")
os.environ["ABM_CONTACTS"]   = str(data_dir / "micro_abm_contacts.csv")
os.environ["ABM30_CONTACTS"] = str(data_dir / "micro_abm_contacts30.csv")
os.environ["WORKPLACE_DAT"]  = str(data_dir / "workplace.dat")
# -------------------------
# Configuration (edit here)
# -------------------------
TARGET_NETWORKS   = ["abm", "DTU", "workplace"]
RAND_BASELINE_R   = 20    # random draws per (run, window) for ΔLCC baseline
MAX_RUNS_PER_ZIP  = 50    # subsample runs for speed; increase if you want
WINDOWS_TO_KEEP   = [1,2,3,4,5]   # days 1..5 only

ROOT = "../"
RESULTS_DIRS = {n: f"{ROOT}/results_{n}" for n in TARGET_NETWORKS}
CACHE_DIRS   = {n: f"{ROOT}/cache_{n}"   for n in TARGET_NETWORKS}

def make_fresh_dir(base):
    stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    out = f"{base}_{stamp}"
    i = 1
    while os.path.exists(out):
        out = f"{base}_{stamp}_{i}"
        i += 1
    os.makedirs(out, exist_ok=False)
    return out

OUT_DIR = make_fresh_dir(f"{ROOT}/dlcc_struct")
print("Writing outputs to:", OUT_DIR)

# Dataset-specific beta/gamma (for filtering)
BG = {
    "DTU":       {"beta": [0.04],  "gamma": [0.005]},
    "abm":       {"beta": [0.01],  "gamma": [0.005]},
    "workplace": {"beta": [0.30],  "gamma": [0.005]},
}

# Filters: only the NPCT variant we care about (ds=1.0, cap=0.25), ii=24, degree risk
def make_filters(network_name):
    return dict(
        variants        = ["without", "dynThresh_FRem"],
        ds              = [1.0],
        fixed_frac      = [0.25, 0.5, None],      
        top_node_frac   = [1.0, None],
        max_rf          = [1.0, None],
        ii              = [24],
        fixed_threshold = [False, True], 
        nprv            = [1],
        risk_model      = ["degree"],
        window          = [24],
        accel_weight    = [1.0],
        pt_weight       = [1.0],
        rise_smoothing  = [1],
        drop_smoothing  = [1],
        beta            = BG[network_name]["beta"],
        gamma           = BG[network_name]["gamma"],
        ibl             = [False],
        compliance      = [1.0],
    )


Writing outputs to: ..//dlcc_struct_20250904_145225


In [2]:
# Cell 2 — Index result ZIPs and load temporal networks (no re-sim)

def index_zip_paths(results_dir):
    """Return dict: parsed key -> zip_path."""
    out = {}
    for zp in glob.glob(os.path.join(results_dir, "results_*.zip")):
        meta = parse_fn(os.path.basename(zp))
        if meta is None:
            continue
        (variant, ii, ds, mrf, frem, tnf, comp, ibl_flag, ft_flag,
         nprv, rm, w, aw, pt, rs, dsp, b, g, is_baseline) = meta
        key = make_key(variant, ii, ds, mrf, frem, tnf, comp, ibl_flag,
                       ft_flag, nprv, rm, w, aw, pt, rs, dsp, b, g)
        out[key] = zp
    return out

zip_index = {n: index_zip_paths(RESULTS_DIRS[n]) for n in TARGET_NETWORKS}
print({n: len(zip_index[n]) for n in TARGET_NETWORKS})

def load_temporal_network(network_name):
    if network_name == "abm":
        return load_network("abm")
    elif network_name == "DTU":
        return load_network("dtu")
    elif network_name == "workplace":
        return load_network("workplace")
    else:
        raise ValueError(f"Unsupported: {network_name}")

# quick sanity: nodes & snapshots
for n in TARGET_NETWORKS:
    temporal = load_temporal_network(n)
    n_nodes = len(get_individuals_from_graph(temporal))
    print(f"{n}: |V|={n_nodes}, T={len(temporal)}")


{'abm': 88, 'DTU': 88, 'workplace': 88}


Loading ABM network:  57%|█████▊    | 138/240 [00:02<00:01, 69.05it/s]

Loading ABM network: 100%|██████████| 240/240 [00:04<00:00, 50.63it/s]


abm: |V|=2058, T=240
Loading DTU graph from /localdata1/dial_mo/NPCT/data/bt_symmetric.csv with temporal_gap=300
DTU: |V|=691, T=240
[LOAD] /localdata1/dial_mo/NPCT/data/workplace.dat – Shape: (9827, 3), Columns: ['Timestamp', 'PersonId1', 'PersonId2']


Loading workplace.dat: 100%|██████████| 275/275 [00:00<00:00, 3768.00it/s]

workplace: |V|=92, T=240


In [3]:
# Cell 3 — Build daily union graphs (structural index) with only what's needed for ΔLCC

def ek(u, v):
    return (u, v) if u <= v else (v, u)

def union_graph_over(temporal, t0, t1):
    H = nx.Graph()
    for t in range(t0+1, t1+1):  # (t0, t1]
        Gt = temporal[min(t, len(temporal)-1)]
        H.add_nodes_from(Gt.nodes())
        H.add_edges_from(Gt.edges())
    return H

def build_daily_index(temporal, interval=24):
    T = len(temporal)
    windows = [(t0, min(t0 + interval, T-1)) for t0 in range(0, T, interval)]
    out = []
    for (t0, t1) in windows:
        H = union_graph_over(temporal, t0, t1)
        if H.number_of_nodes() == 0:
            lcc_size = 0
            edges_all = set()
        else:
            lcc_size = len(max(nx.connected_components(H), key=len)) if H.number_of_edges() else len(H)
            edges_all = set(ek(u, v) for u, v in H.edges())
        out.append(dict(
            t0=t0, t1=t1, H=H,
            edges_all=edges_all,
            lcc_size_pre=lcc_size,
            n_nodes=H.number_of_nodes(),
            n_edges=H.number_of_edges(),
        ))
    return out

# sanity build for one dataset
_tmp = load_temporal_network(TARGET_NETWORKS[0])
demo_idx = build_daily_index(_tmp, interval=24)
print(f"Daily windows built: {len(demo_idx)} (expect ~10 for 10-day nets)")


Loading ABM network:  51%|█████     | 122/240 [00:02<00:02, 48.14it/s]

Loading ABM network: 100%|██████████| 240/240 [00:06<00:00, 34.69it/s]


Daily windows built: 10 (expect ~10 for 10-day nets)


In [4]:
# Cell 4 — Minimal utilities for ΔLCC computation

def removed_pairs_in_window(removed_triplets_window):
    """
    Convert a list of (u, v, t) for a single intervention window into
    a set of undirected {u, v} pairs (ignore timestamps/multiplicity).
    """
    out = set()
    for triple in removed_triplets_window:
        if len(triple) != 3:
            continue
        u, v, _t = triple
        out.add(ek(int(u), int(v)))
    return out

def lcc_size(G):
    if G.number_of_nodes() == 0:
        return 0
    if G.number_of_edges() == 0:
        return 1 if G.number_of_nodes() > 0 else 0
    return len(max(nx.connected_components(G), key=len))

def lcc_after_removing(H, rem_pairs):
    """Largest connected component size after removing edges in rem_pairs."""
    if H.number_of_edges() == 0 or not rem_pairs:
        return lcc_size(H)
    Hp = H.copy()
    to_remove = [(u, v) for (u, v) in rem_pairs if Hp.has_edge(u, v)]
    if to_remove:
        Hp.remove_edges_from(to_remove)
    return lcc_size(Hp)


In [5]:
# Cell 5 — Core: ΔLCC vs random for windows 1..5 only

def analyze_zip_dLCC(
    zip_path,
    structidx,
    max_runs=MAX_RUNS_PER_ZIP,
    R=RAND_BASELINE_R,
    seed=123,
    verbose=True,
    print_every=10,
):
    """
    For each (run, window∈{1..5}), compute:
      - ΔLCC_actual on the union graph H for that window
      - ΔLCC_random: mean/std over R draws with the same edge count (uniform over edges_all)
    Returns one row per (run, window).
    """
    runs = load_runs(zip_path)
    if not isinstance(runs, list):
        raise RuntimeError("Expected list of runs from load_runs(...)")
    runs = runs[:max_runs]

    rng = random.Random(seed)
    rows = []

    nW_total = len(structidx)
    max_w = min(nW_total - 1, max(WINDOWS_TO_KEEP)) if nW_total > 0 else -1
    if max_w < max(WINDOWS_TO_KEEP):
        print(f"      ⚠️ only {nW_total} windows available; clipping to ≤ {max_w}")

    if verbose:
        print(f"      ΔLCC: {len(runs)} runs × windows {WINDOWS_TO_KEEP}")

    for ridx, run in enumerate(runs):
        removed_all_windows = run.get("removed_edges", [])
        nW_run = min(len(removed_all_windows), len(structidx))
        # restrict to requested windows and to those that exist for this run
        windows = [w for w in WINDOWS_TO_KEEP if w < nW_run]

        for w in windows:
            wnd       = structidx[w]
            H         = wnd["H"]
            edges_all = wnd["edges_all"]
            lcc_pre   = wnd["lcc_size_pre"]

            rem_pairs = removed_pairs_in_window(removed_all_windows[w] or [])

            # Actual ΔLCC
            lcc_post = lcc_after_removing(H, rem_pairs)
            dLCC     = lcc_pre - lcc_post

            # Random baseline with same count
            k = len(rem_pairs)
            if k <= 0 or not edges_all:
                dLCC_rand_mean = 0.0
                dLCC_rand_std  = 0.0
            else:
                pool = list(edges_all)
                draws = []
                K = min(k, len(pool))
                for _ in range(R):
                    rnd = set(rng.sample(pool, K))
                    lcc_rand_post = lcc_after_removing(H, rnd)
                    draws.append(lcc_pre - lcc_rand_post)
                dLCC_rand_mean = float(np.mean(draws))
                dLCC_rand_std  = float(np.std(draws))

            rows.append(dict(
                run=ridx, window=w,
                lcc_pre=int(lcc_pre),
                lcc_post=int(lcc_post),
                dLCC=int(dLCC),
                dLCC_rand_mean=dLCC_rand_mean,
                dLCC_rand_std=dLCC_rand_std
            ))

        if verbose and ((ridx + 1) % print_every == 0):
            print(f"        processed {ridx+1}/{len(runs)} runs")

    return pd.DataFrame(rows)


In [6]:
# Cell 6 — Helper: load & filter configs per dataset (dynThresh_FRem only)

def load_df_configs(network_name):
    filters = make_filters(network_name)
    df, _ = load_fixeddepth_summary(
        RESULTS_DIRS[network_name], CACHE_DIRS[network_name], filters
    )
    if df.empty:
        return df

    # Keep dynThresh_FRem only (already in filters, but be explicit)
    df = df[df["variant"] == "dynThresh_FRem"].copy()

    # Map to ZIPs and de-duplicate by zip_path
    zmap = zip_index[network_name]
    df = df[df["_key"].isin(zmap.keys())].copy()
    df["zip_path"] = df["_key"].map(zmap)
    before = len(df)
    df = df.drop_duplicates(subset=["zip_path"]).copy()
    dropped = before - len(df)
    print(f"  {network_name}: {dropped} duplicate config(s) dropped; {len(df)} unique ZIP(s)")
    return df


In [7]:
# Cell Y2 
import os
import numpy as np
import pandas as pd


detail_rows = []

for net in TARGET_NETWORKS:
    df_cfg = load_df_configs(net)
    if df_cfg.empty:
        print(f"[{net}] config missing.")
        continue

    temporal = load_temporal_network(net)
    structidx = build_daily_index(temporal, interval=24)

    for i, row in df_cfg.reset_index(drop=True).iterrows():
        zp  = row["zip_path"]
        cap = None if pd.isna(row["fixed_frac"]) else float(row["fixed_frac"])
        ds  = float(row["drop_strength"])

        dfa = analyze_zip_dLCC(
            zip_path=zp,
            structidx=structidx,
            max_runs=MAX_RUNS_PER_ZIP,
            R=RAND_BASELINE_R,
            seed=123,
            verbose=False,
        )
        if dfa.empty:
            continue

        dfa = dfa[dfa["window"].isin([1,2,3,4,5])].copy()

        day = (
            dfa.groupby("window", as_index=False)
               .agg(mean_lcc_pre=("lcc_pre","mean"),
                    mean_lcc_post_actual=("lcc_post","mean"),
                    mean_dLCC=("dLCC","mean"),
                    mean_rand=("dLCC_rand_mean","mean"))
        )

        day["mean_lcc_post_random"] = day["mean_lcc_pre"] - day["mean_rand"]
        day["lift"] = day["mean_dLCC"] / day["mean_rand"].replace(0, np.nan)

        day["network"] = net
        day["ds"] = ds
        day["cap"] = cap


        day = day[["network","ds","cap","window",
                   "mean_lcc_pre","mean_lcc_post_actual","mean_lcc_post_random",
                   "mean_dLCC","mean_rand","lift"]]

        detail_rows.append(day)


if detail_rows:
    DAY_DETAIL = pd.concat(detail_rows, ignore_index=True)

    # Optional: nur die „wirksamen“ Caps je Datensatz
    PICK = {"abm":0.25, "DTU":0.5, "workplace":0.5}
    mask = DAY_DETAIL.apply(lambda r: np.isclose(r["cap"], PICK.get(r["network"], r["cap"])), axis=1)
    DAY_DETAIL_SEL = DAY_DETAIL[mask].copy().sort_values(["network","window"]).reset_index(drop=True)

    display(DAY_DETAIL_SEL.head(20))

    out_csv = os.path.join(OUT_DIR, "per_day_detailed_table.csv")
    DAY_DETAIL_SEL.to_csv(out_csv, index=False)
    print(f"→ saved: {out_csv}")
else:
    print("no daily data.")


Loading 1 baseline + 0 risk-depth + 2 fixed-depth files


✓ results_dynThresh_FRem_ii24_ds100_mrfNA_frem25_tnfNA_ibl0_c100_ftFalse_nprv1_rmdegree_w24_aw1p0_pt1p0_rs1p0_dsp1p0_b0p01_g0p005.zip (from cache)   (1/3)
✓ results_dynThresh_FRem_ii24_ds100_mrfNA_frem50_tnfNA_ibl0_c100_ftFalse_nprv1_rmdegree_w24_aw1p0_pt1p0_rs1p0_dsp1p0_b0p01_g0p005.zip (from cache)   (2/3)
✓ results_without_ii24_ds100_mrfNA_fremNA_tnfNA_ibl0_c100_ftFalse_nprv1_rmdegree_w24_aw1p0_pt1p0_rs1p0_dsp1p0_b0p01_g0p005_BASELINE.zip (from cache)   (3/3)
  abm: 0 duplicate config(s) dropped; 2 unique ZIP(s)


Loading ABM network: 100%|██████████| 240/240 [00:06<00:00, 37.09it/s]


Loading 1 baseline + 0 risk-depth + 2 fixed-depth files
✓ results_without_ii24_ds100_mrfNA_fremNA_tnfNA_ibl0_c100_ftFalse_nprv1_rmdegree_w24_aw1p0_pt1p0_rs1p0_dsp1p0_b0p04_g0p005_BASELINE.zip (from cache)   (1/3)
✓ results_dynThresh_FRem_ii24_ds100_mrfNA_frem50_tnfNA_ibl0_c100_ftFalse_nprv1_rmdegree_w24_aw1p0_pt1p0_rs1p0_dsp1p0_b0p04_g0p005.zip (from cache)   (2/3)
✓ results_dynThresh_FRem_ii24_ds100_mrfNA_frem25_tnfNA_ibl0_c100_ftFalse_nprv1_rmdegree_w24_aw1p0_pt1p0_rs1p0_dsp1p0_b0p04_g0p005.zip (from cache)   (3/3)
  DTU: 0 duplicate config(s) dropped; 2 unique ZIP(s)
Loading DTU graph from /localdata1/dial_mo/NPCT/data/bt_symmetric.csv with temporal_gap=300
Loading 1 baseline + 0 risk-depth + 2 fixed-depth files
✓ results_dynThresh_FRem_ii24_ds100_mrfNA_frem25_tnfNA_ibl0_c100_ftFalse_nprv1_rmdegree_w24_aw1p0_pt1p0_rs1p0_dsp1p0_b0p3_g0p005.zip (from cache)   (1/3)
✓ results_dynThresh_FRem_ii24_ds100_mrfNA_frem50_tnfNA_ibl0_c100_ftFalse_nprv1_rmdegree_w24_aw1p0_pt1p0_rs1p0_dsp1p0_b0p3

Loading workplace.dat: 100%|██████████| 275/275 [00:00<00:00, 3078.10it/s]


,network,ds,cap,window,mean_lcc_pre,mean_lcc_post_actual,mean_lcc_post_random,mean_dLCC,mean_rand,lift
0,DTU,1.0,0.50,1,500.0,470.98,475.202,29.02,24.798,1.170256
1,DTU,1.0,0.50,2,486.0,341.58,375.867,144.42,110.133,1.311324
2,DTU,1.0,0.50,3,473.0,468.78,467.317,4.22,5.683,0.742566
3,DTU,1.0,0.50,4,267.0,141.56,141.589,125.44,125.411,1.000231
4,DTU,1.0,0.50,5,331.0,331.00,331.000,0.00,0.000,NaN
5,abm,1.0,0.25,1,1925.0,1920.12,1919.628,4.88,5.372,0.908414
6,abm,1.0,0.25,2,1896.0,1702.90,1794.473,193.10,101.527,1.901957
7,abm,1.0,0.25,3,1917.0,1737.60,1825.816,179.40,91.184,1.967450
8,abm,1.0,0.25,4,1923.0,1754.00,1838.345,169.00,84.655,1.996338
9,abm,1.0,0.25,5,1552.0,988.80,1394.587,563.20,157.413,3.577849


→ saved: ..//dlcc_struct_20250904_145225/per_day_detailed_table.csv
